In [ ]:
# Basic packages
import json
import glob
import re
from os.path import exists
from datetime import datetime
from random import sample, seed

save_directory = "/u/scr/nlp/data/nli-consistency/vqa-camera" # Instructions for user : indicate directory within which to save train data

# Data from https://visualgenome.org/api/v0/api_home.html
# Used Version 1.2 as that one had Part 1 and Part 2
VG_path = '/u/scr/nlp/data/nli-consistency/vg_data/VG_100K/' # Instructions for user: indicate path
VG_path2 = '/u/scr/nlp/data/nli-consistency/vg_data/VG_100K_2/'

# L-ConVQA from https://arijitray1993.github.io/ConVQA/
file_paths = glob.glob('/u/scr/nlp/data/nli-consistency/dataset_MTURK_refine/*') # Instructions for user: indicate path
file_names = [re.search(r'qas_(.*?)\.json', path).group(1) for path in file_paths] # Gets the image number
indices = [i for i in range(len(file_names))]

In [ ]:
rand_seed=1
seed(rand_seed)
num_images = 10000 # There are a total of ~100,000 images

selected_questions = []

assert len(selected_questions) <= num_images

counter = 0

while len(selected_questions) < num_images:
    new_questions = sample(indices, num_images)
    for idx in new_questions:
        counter += 1
        file_path = file_paths[idx]
        image_num = file_names[idx]
        
        if file_path in selected_questions:
            continue
        
        image_path1 = VG_path + str(image_num) + '.jpg'
        image_path2 = VG_path2 + str(image_num) + '.jpg'

        # image must exist
        if exists(image_path1):
            image_path = image_path1
        elif exists(image_path2):
            image_path = image_path2
        else:
            print("Error: Image does not exist")
            continue
            
        # file must have questions
        has_questions = False
        with open(file_path, 'r') as file_stream:
            qas = json.load(file_stream)['consistent']
        
        for jdx in range(len(qas)):
            if len(qas[jdx]) >= 2:
                has_questions = True
                selected_questions.append(file_path)
                break
#         if not has_questions:
#             print("Images considered: %s", counter)
#             print("Error: Image %s does not have questions" % image_num)
#             print(qas)
        
        if len(selected_questions) == num_images:
            break

if len(selected_questions) != len(set(selected_questions)):
    print("Error: There was a problem sampling the questions properly; there are non-unique file paths")
            
curr_time = datetime.now()
curr_time_formatted = curr_time.strftime("%Y-%m-%d-%H:%M:%S")

file_store_name = "{}/vg-data-".format(save_directory) + curr_time_formatted + "-n=" + str(len(selected_questions)) + "-seed=" + str(rand_seed) + ".txt"

with open(file_store_name, 'w') as filehandle:
    for listitem in selected_questions:
        filehandle.write('%s\n' % listitem)